In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pandas as pd
import numpy as np

In [3]:
trainDf = pd.read_csv('./train.csv').drop(['Id'], axis=1)[:500]
testDf = pd.read_csv('./test.csv').drop(['Id'], axis=1)[:500]

In [4]:
trainDf.head()

,Comment,Topic
0,A few things. You might have negative- frequen...,Biology
1,Is it so hard to believe that there exist part...,Physics
2,There are bees,Biology
3,I'm a medication technician. And that's alot o...,Biology
4,Cesium is such a pretty metal.,Chemistry


In [5]:
testDf.head()

,Comment,Topic
0,Personally I have no idea what my IQ is. I’ve ...,Biology
1,I'm skeptical. A heavier lid would be needed t...,Physics
2,I think I have 100 cm of books on the subject....,Biology
3,Is chemistry hard in uni. Ive read somewhere t...,Chemistry
4,"In addition to the other comment, you can crit...",Physics


In [6]:
# update column with condition
trainDf.loc[trainDf['Topic'] =='Biology', 'Topic'] = 0
trainDf.loc[trainDf['Topic'] =='Physics', 'Topic'] = 1
trainDf.loc[trainDf['Topic'] =='Chemistry', 'Topic'] = 2
trainDf.head()

,Comment,Topic
0,A few things. You might have negative- frequen...,0
1,Is it so hard to believe that there exist part...,1
2,There are bees,0
3,I'm a medication technician. And that's alot o...,0
4,Cesium is such a pretty metal.,2


In [7]:
testDf.loc[testDf['Topic'] =='Biology', 'Topic'] = 0
testDf.loc[testDf['Topic'] =='Physics', 'Topic'] = 1
testDf.loc[testDf['Topic'] =='Chemistry', 'Topic'] = 2
testDf.head()

,Comment,Topic
0,Personally I have no idea what my IQ is. I’ve ...,0
1,I'm skeptical. A heavier lid would be needed t...,1
2,I think I have 100 cm of books on the subject....,0
3,Is chemistry hard in uni. Ive read somewhere t...,2
4,"In addition to the other comment, you can crit...",1


In [8]:
# copy data to temp
temp = trainDf.copy()
temp['len'] = temp['Comment'].apply(lambda x: len(x.split()))
# sort by len desc
temp.sort_values(by=['len'], ascending=False).head()

,Comment,Topic,len
355,Here’s a lengthy quote where you rejected that...,1,1274
209,"Hirsch isn't looking to ""recreate the plots ex...",1,263
285,"I agree with most of what you said up there, t...",1,259
0,A few things. You might have negative- frequen...,0,254
183,I think we've converged on most points.\n\nFor...,1,246


In [9]:
# Instantiate the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
# Create lists to store the train and test data
train_data = []
test_data = []

In [11]:
MAX_LEN = 512
# MAX_LEN = 1024
#  Iterate through the dataset
for index, row in trainDf.iterrows():
    # Tokenize the text
    text = row['Comment']
    
    # tokens = tokenizer.tokenize(text)
    tokens = tokenizer.encode_plus(text, None, add_special_tokens=True, max_length=MAX_LEN, padding='max_length', truncation=True, return_attention_mask=True, return_tensors='pt')
    # print(tokens)
    # inputs.append(tokenizer.convert_tokens_to_ids(tokens))
    # inputs.append({
    #     'input_ids': tokens['input_ids'],
    #     'attention_mask': tokens['attention_mask'],
    #     'token_type_ids': tokens['token_type_ids'].flatten()
    # })
    train_data.append({
        'input_ids': tokens['input_ids'].flatten(),
        'attention_mask': tokens['attention_mask'].flatten(),
        'token_type_ids': tokens['token_type_ids'].flatten(),
        'labels': torch.tensor(row['Topic'])
    })
    # Convert the label to a numerical value
    # labels.append(row['Topic'])
    # label = row['Topic']
    # if label == 'a':
    #     labels.append(0)
    # elif label == 'b':
    #     labels.append(1)
    # elif label == 'c':
    #     labels.append(2)
    # else:
    #     labels.append(3)

'''For test dataset'''
for index, row in testDf.iterrows():
    # Tokenize the text
    text = row['Comment']
    
    # tokens = tokenizer.tokenize(text)
    tokens = tokenizer.encode_plus(text, None, add_special_tokens=True, max_length=MAX_LEN, padding='max_length', return_attention_mask=True, return_tensors='pt')

    test_data.append({
        'input_ids': tokens['input_ids'].flatten(),
        'attention_mask': tokens['attention_mask'].flatten(),
        'token_type_ids': tokens['token_type_ids'].flatten(),
        'labels': torch.tensor(row['Topic'])
    })

In [12]:
# train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)

# for batch_idx, data in enumerate(train_data_loader):
# # for data in train_data:
#     input_ids = data['input_ids']
#     attention_mask = data['attention_mask']
#     token_type_ids = data['token_type_ids']
#     labels = data['labels']

#     # if input_ids.shape[0] != MAX_LEN or attention_mask.shape[0] != MAX_LEN or token_type_ids.shape[0] != MAX_LEN:
#     #     print('\n\n\n\n\n')
#     #     print(input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)
#     #     print('\n\n\n\n\n')
#     # else:
#     #     print(input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)
#     # print(input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)

In [13]:
# Convert the input ids and labels to tensors
# inputs_tensor = torch.tensor(inputs)
# labels_tensor = torch.tensor(labels)

In [14]:
torch.cuda.is_available()

True

In [15]:
# empty cache
torch.cuda.empty_cache()

# Instantiate the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Move the model to the GPU
model.to('cuda')

# Set the model to training mode
model.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:
# Define the loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Define the number of training steps
# num_steps = 1000
num_steps = 15

# batch size
batch_size = 8


In [17]:
# inputs_tensor = inputs

In [18]:
# len(inputs_tensor)
# inputs_tensor[0]

In [19]:
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
# for idx, data in enumerate(train_data_loader):
#     temp = data['input_ids']
#     for i in data['input_ids']:
#         print(i)
#     break
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [20]:
torch.cuda.empty_cache()
val_targets = []
val_outputs = []

In [21]:
# Iterate over the training steps
for step in range(num_steps):
    
    train_loss = 0
    valid_loss = 0
    # print("Step: {}, Train_Loss: {}, val_Loss {}".format(step, train_loss, valid_loss))
    '''Train portion'''
    model.train()
    for batch_idx, data in enumerate(train_data_loader):
        # Get the inputs and labels
        input_ids = data['input_ids']
        attention_mask = data['attention_mask']
        token_type_ids = data['token_type_ids']
        labels = data['labels']

        # Move the inputs and labels to the GPU
        input_ids = input_ids.to('cuda')
        attention_mask = attention_mask.to('cuda')
        token_type_ids = token_type_ids.to('cuda')
        labels = labels.to('cuda')

        # print(input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)

        # Clear the gradients
        optimizer.zero_grad()

        # Forward pass
        # print(input_ids)
        # print(attention_mask)
        # print(token_type_ids)
        outputs = model(input_ids, attention_mask, token_type_ids)
        # print(outputs[0])
        logits = outputs[0]
        loss = loss_fn(logits, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        
    '''Test portion'''
    model.eval()
    for batch_idx, data in enumerate(train_data_loader):
        # Get the inputs and labels
        input_ids = data['input_ids']
        attention_mask = data['attention_mask']
        token_type_ids = data['token_type_ids']
        labels = data['labels']

        # Move the inputs and labels to the GPU
        input_ids = input_ids.to('cuda')
        attention_mask = attention_mask.to('cuda')
        token_type_ids = token_type_ids.to('cuda')
        labels = labels.to('cuda')

        outputs = model(input_ids, attention_mask, token_type_ids)
        logits = outputs[0]

        loss = loss_fn(logits, labels)
        valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
        val_targets.extend(labels.cpu().detach().numpy().tolist())
        # val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    # Print the loss
    # if step % 100 == 0:
    #     print("Step: {}, Loss: {}".format(step, loss.item()))
    print("Step: {}, Train_Loss: {}, val_Loss {}".format(step, train_loss, valid_loss))

Step: 0, Train_Loss: 1.0837634839708843, val_Loss 1.0280371894912117
Step: 1, Train_Loss: 0.9539796747858563, val_Loss 0.728902604844835
Step: 2, Train_Loss: 0.678503724790755, val_Loss 0.44708701778971954
Step: 3, Train_Loss: 0.42422813389982506, val_Loss 0.23765368925200572
Step: 4, Train_Loss: 0.26186059521777305, val_Loss 0.1151672158212889
Step: 5, Train_Loss: 0.13861566279379145, val_Loss 0.07046252551178135
Step: 6, Train_Loss: 0.10147081883180713, val_Loss 0.05474041922697944
Step: 7, Train_Loss: 0.06237984218058133, val_Loss 0.035615112809907826
Step: 8, Train_Loss: 0.05648477730296907, val_Loss 0.03204192827263521
Step: 9, Train_Loss: 0.03902717339732344, val_Loss 0.026739070465463016
Step: 10, Train_Loss: 0.03065002374794512, val_Loss 0.022417416019986078
Step: 11, Train_Loss: 0.0288210812187384, val_Loss 0.02111445986000555
Step: 12, Train_Loss: 0.028396989758466447, val_Loss 0.020226761284801692
Step: 13, Train_Loss: 0.025249478296332416, val_Loss 0.019321485559293444
Step

In [22]:
# for batch_idx, data in enumerate(train_data_loader):
# len(train_data_loader)
# train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
# print(len(train_data), len(train_data_loader))

In [54]:
y_pred = []
y_true = []

label_list = ['Biology', 'Physics', 'Chemistry']
for i in range(500):
    test_text = testDf.iloc[30]
    # print(test_text['Comment'])
    # print(label_list[test_text['Topic']])
    y_true.append(test_text['Topic'])

    encodings = tokenizer.encode_plus(
        test_text['Comment'],
        None,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    model.eval()
    with torch.no_grad():
        input_ids = encodings['input_ids'].to('cuda', dtype=torch.long)
        attention_mask = encodings['attention_mask'].to('cuda', dtype=torch.long)
        token_type_ids = encodings['token_type_ids'].to('cuda', dtype=torch.long)
        output = model(input_ids, attention_mask, token_type_ids)
        final_output = torch.sigmoid(output[0]).cpu().detach().numpy().tolist()
        # print(label_list[int(np.argmax(final_output, axis=1))])
        y_pred.append(int(np.argmax(final_output, axis=1)))

In [55]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true, y_pred)

1.0